In [91]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')   
result = pd.read_excel("Test_results_for_POC_21092018.xlsx",parse_dates=[['Rn Execution Date', 'Rn Execution Time']])
result = result.rename(columns={'Rn Execution Date_Rn Execution Time':'date'})
defect = pd.read_excel("Defect_results_for_POC_21092018.xlsx")
test = pd.read_excel("Testcases_CTC_MS_for_POC_27092018.xlsx")
result.columns = result.columns.str.replace(' ', '_')
defect.columns = defect.columns.str.replace(' ', '_')
test.columns = test.columns.str.replace(' ', '_')

### Merge from test result and CTC

In [45]:
def add_soft_ver_count_column(result) :
    '''adds count column in test result dataframe and returns dataframe.'''
    group = result.groupby(['Ts_User_08_(Test_ID)', 'Cy_Terminal_SW_Version'])
    df1 = group.Ts_Name.agg('count').reset_index()
    df1 = df1.rename(columns={'Ts_Name':'soft_ver_count'})
    result = pd.merge(result,df1,how='outer',on=['Ts_User_08_(Test_ID)','Cy_Terminal_SW_Version'])
    return result
result = add_soft_ver_count_column(result)
#result.to_excel('df1.xlsx',index=False)

In [46]:
def count_pass_fail_ratio(master) :
    master = master.rename(columns={'Ts_User_08_(Test_ID)': 'test_id'})
    p4 = master.groupby(['test_id','Rn_Status'])
    df = p4.Ts_Name.agg('count').reset_index()
    df = df.rename(columns={'Ts_Name': 'count'})
    df = df[df['count'] > 0]
    pass_df = df[df['Rn_Status'] == 'Passed']
    fail_df = df[df['Rn_Status'] == 'Failed']
    df4 = pd.merge(pass_df,fail_df,how='outer',on=['test_id'])
    df4 = df4.rename(columns={'count_x': 'pass_count','count_y':'fail_count'})
    df4['pass_count'].fillna(0,inplace=True)
    df4['fail_count'].fillna(0,inplace=True)
    df4['pass_ratio'] = df4['pass_count']/(df4['pass_count']+ df4['fail_count'])
    df4['fail_ratio'] = 1 - df4['pass_ratio']
    df4.drop(labels=['Rn_Status_x', 'Rn_Status_y'], axis=1,inplace=True)
    master = pd.merge(master,df4,how='left',on=['test_id'])
    master = master.rename(columns={'test_id':'Ts_User_08_(Test_ID)'})
    return master
result = count_pass_fail_ratio(result)

In [48]:
# if ran two times it will throw error.find out why ?
def add_defect_count(df) :
    '''input defect sheet.output adds a column to defect sheet.returns defect sheet with added column.'''
    group = df.groupby(['Linked_Test_ID'])
    df1 = group.Defect_ID.agg('count').reset_index()
    df1 = df1.rename(columns={'Defect_ID':'total_defect_count'})
    df = pd.merge(df,df1,how='outer',on=['Linked_Test_ID'])
    df['total_defect_count'].fillna(0,inplace=True)
    return df
defect = add_defect_count(defect)
#defect.to_excel('df.xlsx',index=False)

In [49]:
def add_pass_defect_count(df) :
    '''input defect sheet.output adds a column to defect sheet.returns defect sheet with added column.'''
    df2 = df[(df['Status'] == 'Open') | (df['Status'] == 'Reopen') | (df['Status'] == 'Retest')] 
    group = df2.groupby(['Linked_Test_ID'])
    df1 = group.Defect_ID.agg('count').reset_index()
    df1 = df1.rename(columns={'Defect_ID':'open_defect_count'})
    df = pd.merge(df2,df1,how='outer',on=['Linked_Test_ID'])
    df['open_defect_count'].fillna(0,inplace=True)
    return df
defect = add_pass_defect_count(defect)

In [50]:
defect.drop(labels=['Test_Area', 'Summary','Description', 'Full_Defect_Comments', 'Also_in_OpCo', 'Detected_By'],
           axis=1,inplace=True)
defect = defect.rename(columns={'Linked_Test_ID':'Ts_User_08_(Test_ID)'})

In [54]:
all_tc = all_tc.sort_values('date')
all_tc.drop_duplicates(subset=['Ts_User_08_(Test_ID)','Cy_Terminal_SW_Version','Cy_Executing_OpCo'], keep='last', inplace=True)
all_tc.shape
raw_data.to_excel('raw.xlsx',index=False)

(22409, 33)

In [55]:
all_tc.to_excel('All_TC.xlsx',index=False)

### Reading RN and creating dict

In [92]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')   
Alltestcases = pd.read_excel("All_TC.xlsx")
RE_output = pd.read_excel('FinalTestSuite_nonOptimized.xlsx')

In [93]:
Alltestcases.rename(columns={'Ts_User_08_(Test_ID)':'test_id'},inplace = True)

In [94]:
RE_output.rename(columns={'Ts_User_08_(Test_ID)':'test_id'},inplace = True)

In [95]:
def take_release_notes(file1) :
    '''modified version of take_latest_release_note for internal use only.
    returns dataframe containing given release notes.'''
    #file1 = input('Please enter excel file names of latest release : ' )       
    if(not (file1.endswith('.xls') or file1.endswith('.xlsx'))) :
        file1 = file1+'.xlsx'
    if(not(os.path.isfile(file1))) :
        print('no file found for given name : '+file1)
        take_release_notes()
    row1 = pd.read_excel(file1)
    if(row1.empty) :
        print('Given excel file is empty')
        take_release_notes()
    if(row1.duplicated(subset=' Release Notes Name').any() == True) :
        print('Given file contains duplicate column names.Please update it.')
        take_release_notes()
    df = row1.set_index(' Release Notes Name').T
    return df
RN = take_release_notes('Release notes_Hima_TA2_DSIM')

### Condition -1 

In [96]:
soft_ver = RN['Software Version'].values[0].strip()
Alltestcases['is_selected'] = np.where(((Alltestcases['Cy_Terminal_SW_Version'] == soft_ver) &  ( Alltestcases['Cy_Executing_OpCo'] == 'CTC' )), 1, 0)

In [97]:
Alltestcases['is_selected'].value_counts()

0    22170
1      239
Name: is_selected, dtype: int64

### Condition -2 

In [84]:
Alltestcases['is_selected'] = Alltestcases['test_id'].isin(RE_output['test_id'])

In [87]:
Alltestcases['is_selected'].value_counts()

False    14047
True      8362
Name: is_selected, dtype: int64

In [98]:
dict1 = {'Defect': [25731, 25706, 25717, 25724], 'known_issue': [], 'service_name': ['Advanced LTE DL'], 
         'imei_sv': ['IMEI-SV'], 'release_type': ['TA'], 'terminal': 'Hima'}
dict2 =  {'Defect': [25670, 25717, 25719, 25722, 25728], 'service_name': [], 
          'imei_sv': ['IMEI-SV'], 'release_type': ['MR0'], 'terminal': 'Hima'}

In [99]:
def add_columns_from_input(dict,df) :
    '''pass dictionary we are getting from RN_diff function. and master dataframe in which you want to add the columns.
    Return dataframe with added columns.'''
    defect_list = dict.get('Defect')
    col_name = 'defect_from_RN'
    if len(defect_list)==0:
        df['defect_from_RN']= 0
    else:
        for i,defect in enumerate(defect_list) :
            df.loc[df['Defect_ID'] == defect, col_name] = defect
        df[col_name].fillna(0,inplace=True)
    area_list = dict.get('service_name')
    imei_sv = dict.get('imei_sv')
    if(len(imei_sv) == 1 ) :
        area_list.extend(imei_sv)
    col_name = 'area_from_RN'
    for i,area in enumerate(area_list) :
        df.loc[df['Ts_Test_Area'] == area, col_name] = area
    df[col_name].fillna('missing',inplace=True)
    df['input_terminal'] = str(dict.get('terminal'))
    df['input_release'] = str(dict.get('release_type')[0])
    return df
Alltestcases = add_columns_from_input(dict1,Alltestcases)

In [100]:
Alltestcases.to_excel('master1_old.xlsx',index=False)